In [1]:
from Bio import SeqIO
import pandas as pd
import numpy as np
import re
import os
import random
import subprocess
from collections import defaultdict
from tqdm import tqdm
from collections import Counter
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
PATH = "/home/mchernigovskaya/data/paired/10x"

In [3]:
bcr = ['IGH', 'IGL', 'IGK']
tcr = ['TRA', 'TRB', 'TRG']
trash = ['Multi', 'None']

In [4]:
# help functions

def dict_to_sorted_df_with_perc(dict_name, column_name):
    df = dict_to_sorted_df(dict_name, ['# ' + column_name])
    df = add_perc_to_df(df, column_name)
    return df

def dict_to_sorted_df(dict_name, column_labels):
    df = pd.DataFrame.from_dict(dict_name, orient='index')
    df.columns = column_labels
    return df.sort_index(0)

def add_perc_to_df(dict_name, column_name):
    dict_name['% ' + column_name] = dict_name['# '+column_name] / sum(dict_name['# '+column_name]) * 100
    return dict_name

def get_counter_dict(v):
    return dict(Counter(v))

In [5]:
def get_info_by_chain_type(filtered_contigs_info, chain_type):
    return filtered_contigs_info.loc[filtered_contigs_info['chain'].isin(chain_type)]
    

def chains_simple_stats(filtered_contigs_info):
    
    # dict: chain_type -> number of contigs
    contigs_types = {chain_type: len(rows) for chain_type, rows in filtered_contigs_info.groupby('chain')}
    
    stats_by_contig_types = dict_to_sorted_df_with_perc(contigs_types, 'contigs')
    
    def summary_from_stats_by_contig_types(chain_type):
        return sum(stats_by_contig_types.loc[stats_by_contig_types.index.isin(chain_type)]['% contigs'])
    
    summary = dict_to_sorted_df({"bcr" : [summary_from_stats_by_contig_types(bcr), "ok"], 
           "tcr" : [summary_from_stats_by_contig_types(tcr), "filter out"], 
           "trash" : [summary_from_stats_by_contig_types(trash), "igblast for fun + filter out"]}, ["% contigs", "TODO"])
    
    # print stats
    print("> Number of all contigs: {}\n".format(sum(stats_by_contig_types['# contigs'])))
    
    print("> Contigs by chain types:\n")
    print(stats_by_contig_types)
    print("\n(A value of Multi indicates that segments from multiple chains were present)")
    
    print("\n> Summary:\n")
    print(summary)
    

def cells_simple_stats(bcr_contigs_info):
    
    # dict : cell_barcode -> number of bcr chains
    cells_types = {cell_barcode : len(rows) for cell_barcode, rows in bcr_contigs_info.groupby('barcode')}
    
    summary = dict_to_sorted_df_with_perc(get_counter_dict(cells_types.values()), 'cells')
    summary.index = [str(name) + " chains" for name in summary.index]
     
    # dict: cell_barcode -> counts of bcr chains 
    chains_counts_by_cell = {cell_barcode : str(get_counter_dict(rows['chain'])) for cell_barcode, rows in bcr_contigs_info.groupby('barcode')}
    accurate_counts = dict_to_sorted_df_with_perc(get_counter_dict(chains_counts_by_cell.values()), 'cells')
    accurate_counts = accurate_counts.sort_values(by='# cells',  ascending=False)
    
    
    # print stats
    print("> Number of all cells: {}\n".format(len(cells_types)))
    
    print("> Cells by number of all chains\n")
    print(summary)
    
    print("\n> Cells by combination of chains\n")
    print(accurate_counts)
    

def trash_simple_stats(trash_contigs_info):
    print("Looks like we should use treshold = 10")
    
    umis_counts = dict_to_sorted_df_with_perc(get_counter_dict(trash_contigs_info['umis']), 'umis')
    print(umis_counts)

    
def plot_umis_distribution(bcr_contigs_info):
    plt.hist(bcr_contigs_info['umis'], bins=range(0, 100), range=(0, 100))  

    
def filter_umis(contigs_info, treshold):
    return contigs_info.loc[contigs_info['umis'] > treshold]


def report_filtered_bcrs(filtered_bcr_contigs_info, contigs_fname):
    filtered_contig_id = set(filtered_bcr_contigs_info['contig_id'])
    contigs_fname_out = contigs_fname[:-6] + '_filtered.fasta' 
    open(contigs_fname_out, 'w').close()
    with open(contigs_fname_out, 'a') as f:
        for rec in SeqIO.parse(contigs_fname, "fasta"):
            if rec.id in filtered_contig_id: 
                SeqIO.write(rec, f, "fasta")
    print('\nFiltered bcrs contigs were written to {}\n'.format(contigs_fname_out))


In [6]:
def process_filtered_contigs_csv(dataset_name):
    
    annotation_fname = PATH + '/{}/vdj_v1_{}_b_filtered_contig_annotations.csv'.format(dataset_name, dataset_name)
    
    # read filtered_contig_annotations.csv
    filtered_contigs_info = pd.read_csv(annotation_fname, delimiter=',')
    filtered_contigs_info = filter_umis(filtered_contigs_info, 10)
    
    print("\n\n=========== contigs ===========\n\n")
    chains_simple_stats(filtered_contigs_info)

    print("\n\n=========== bcr chains ===========\n\n")    
    bcr_contigs_info = get_info_by_chain_type(filtered_contigs_info, bcr)   
    
    cells_simple_stats(bcr_contigs_info)
    
#     print("\n\n=========== trash chains coverage ===========\n\n")
#     trash_contigs_info = get_info_by_chain_type(filtered_contigs_info, trash)
#     trash_simple_stats(trash_contigs_info)
    
#     plot_umis_distribution(bcr_contigs_info)

    contigs_fname = PATH + '/{}/vdj_v1_{}_b_filtered_contig.fasta'.format(dataset_name, dataset_name)
    report_filtered_bcrs(bcr_contigs_info, contigs_fname)
    
    

# CD19

CD19+ B cells isolated from peripheral blood mononuclear cells (PBMCs) of a healthy donor, purchased from AllCells (Catalog #: PB010-0), 93% viable by Trypan blue stain.

CD19+ B cells are primary cells with relatively small amounts of RNA (~1pg RNA/cell).

Libraries were prepared following the Single Cell V(D)J Reagent Kits User Guide (CG000086 RevC).



In [7]:
process_filtered_contigs_csv('cd19')



=========== contigs ===========


> Number of all contigs: 18407

> Contigs by chain types:

       # contigs  % contigs
IGH         7518  40.843157
IGK         6435  34.959526
IGL         4267  23.181398
Multi        183   0.994187
TRA            1   0.005433
TRB            3   0.016298

(A value of Multi indicates that segments from multiple chains were present)

> Summary:

       % contigs                          TODO
bcr    98.984082                            ok
tcr     0.021731                    filter out
trash   0.994187  igblast for fun + filter out


=========== bcr chains ===========


> Number of all cells: 9188

> Cells by number of all chains

          # cells    % cells
1 chains     2024  22.028733
2 chains     5749  62.570744
3 chains     1066  11.602090
4 chains      264   2.873313
5 chains       71   0.772747
6 chains       10   0.108838
7 chains        3   0.032651
8 chains        1   0.010884

> Cells by combination of chains

                                #

# GM12878

B-lymphoblastoid cell line GM12878, purchased from Coriell (Catalog #: GM12878), 87% viable by Trypan blue stain.

GM12878 cells are a B-lymphoblastoid cell line with high expression level of Ig transcripts (mostly IGHM/D and IGL isotype).

Libraries were prepared following the Single Cell V(D)J Reagent Kits User Guide (CG000086 RevC).

In [8]:
process_filtered_contigs_csv('gm12878')



=========== contigs ===========


> Number of all contigs: 1882

> Contigs by chain types:

       # contigs  % contigs
IGH          735  39.054198
IGK            6   0.318810
IGL         1051  55.844846
Multi         87   4.622742
TRB            3   0.159405

(A value of Multi indicates that segments from multiple chains were present)

> Summary:

       % contigs                          TODO
bcr    95.217853                            ok
tcr     0.159405                    filter out
trash   4.622742  igblast for fun + filter out


=========== bcr chains ===========


> Number of all cells: 845

> Cells by number of all chains

          # cells    % cells
1 chains      151  17.869822
2 chains      477  56.449704
3 chains      183  21.656805
4 chains       32   3.786982
5 chains        2   0.236686

> Cells by combination of chains

                      # cells    % cells
{'IGL': 1, 'IGH': 1}      408  48.284024
{'IGL': 2, 'IGH': 1}      170  20.118343
{'IGL': 1}                 

# PBMC

Peripheral blood mononuclear cells (PBMCs) from a healthy donor, purchased from AllCells (Catalog #: PB001). >90% viable by Trypan blue stain. CD3+ population is estimated ~50% by FACs.

PBMCs are primary cells with relatively small amounts of RNA (~1pg RNA/cell).

In [15]:
process_filtered_contigs_csv('pbmc')



=========== contigs ===========


> Number of all contigs: 718

> Contigs by chain types:

       # contigs  % contigs
Multi          4   0.557103
TRA           49   6.824513
TRB          665  92.618384

(A value of Multi indicates that segments from multiple chains were present)

> Summary:

       % contigs                          TODO
bcr     0.000000                            ok
tcr    99.442897                    filter out
trash   0.557103  igblast for fun + filter out


# tree visualizer hack